# DETOXIFICATION

In [1]:
%pip install -q torch
%pip install -q transformers[torch]
%pip install -q googletrans==3.1.0a0
%pip install -q fastparquet
%pip install sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.1.143 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.54.4 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 M

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Lectura de datos

Vamos a leer el dataset de entrenamiento. Este dataset esta formado por 400 frases tóxicas por idioma en 9 idiomas diferentes y sus versiones no tóxicas. Formaremos un dataset conjunto que contenga para cada frase, su idioma de origen, la frase tóxica y la frase de-toxificada.

In [ ]:
splits = {'en': 'data/en-00000-of-00001.parquet', 'ru': 'data/ru-00000-of-00001.parquet', 'uk': 'data/uk-00000-of-00001.parquet', 'de': 'data/de-00000-of-00001.parquet', 'es': 'data/es-00000-of-00001.parquet', 'am': 'data/am-00000-of-00001.parquet', 'zh': 'data/zh-00000-of-00001.parquet', 'ar': 'data/ar-00000-of-00001.parquet', 'hi': 'data/hi-00000-of-00001.parquet'}
df_en = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["en"])
df_uk = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["uk"])
df_ru = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ru"])
df_de = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["de"])
df_es = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["es"])
df_am = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["am"])
df_zh = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["zh"])
df_ar = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ar"])
df_hi = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["hi"])

df_hf = pd.concat([df_en, df_uk, df_ru, df_de, df_es, df_am, df_zh, df_ar, df_hi], keys=['en','uk', 'ru', 'de', 'es', 'am', 'zh-cn', 'ar', 'hi']).reset_index(level=[0])
df_hf.sample(3)

El siguiente dataset es de github, contiene 500 ejemplos en inglés (de momento lo uso de test, hasta que se implemente la traduccion con el de arriba)

In [2]:
df_gh = pd.read_csv("https://raw.githubusercontent.com/panlingua/multilingual_text_detoxification_datasets/refs/heads/main/refined-english/en_parallel_detoxification.test.csv")
df_gh.rename(columns={"toxic_comment":"toxic_sentence", "civil_comment":"neutral_sentence"}, inplace=True)
df_gh.sample(3)

,ID,toxic_sentence,neutral_sentence
62,1212,"if you want to get fucked , try grindr .","If you want a date ,try grindr"
267,941,i 'm so fucking sick of reddit assuming people...,I'm annoyed that Reddit is assuming people wit...
230,2720,"you lost bitches , now stfu .","you lost, admit it and calm down."


dataset que nos da la tarea

In [3]:
df_tarea=pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")
df_tarea.rename(columns={"en_toxic_comment":"toxic_sentence", "en_neutral_comment":"neutral_sentence"}, inplace=True)
df_tarea.sample(3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,toxic_sentence,neutral_sentence
3962,her hair is always in her way and she 's just ...,her hair is always in her way and she's insuff...
11273,now that the shit is hitting the fan someone h...,Now that the issue is hitting the fan someone ...
3633,hard to believe someone can be that ignorant .,It is hard to believe that someone can be that...


📝 Finalmente, se define el **dataset final** que utilizará los dataframes anteriormente implementados.

### Train-test split

In [4]:
train, val = train_test_split(df_tarea, train_size=0.8, random_state=33)

X_train = list(train['toxic_sentence'])
y_train = list(train['neutral_sentence'])

X_val = list(val['toxic_sentence'])
y_val = list(val['neutral_sentence'])

X_test = list(df_gh['toxic_sentence'])
y_test = list(df_gh['neutral_sentence'])
for i,e in enumerate(y_test):
    if not isinstance(e, str):
        X_test.pop(i)
        y_test.pop(i)

## Pipeline de detoxificación

Nuestro modelo de detoxificación tendrá un pipeline que consistirá de los siguientes pasos:

1. **Traducción**: Traduciremos las frases del idioma original al inglés
2. **Detoxificación**: El modelo detoxificará la frase en inglés
3. **Retro-traducción**: La frase detoxificada será traducida del inglés al idioma original

## Detoxificación

Ahora haremos un finetunning de un modelo de Huggingface para detoxificar el código.

#### Creación de los dataset

In [5]:
class DetoxificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        # Reemplazar pad_token_id con -100 para ignorar tokens de relleno en la pérdida
        # item["labels"][item["labels"] == tokenizer.pad_token_id] = -100

        return item

In [6]:
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

X_train_tokens = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
y_train_tokens = tokenizer(y_train, truncation=True, padding=True, return_tensors="pt")

X_val_tokens = tokenizer(X_val, truncation=True, padding=True, return_tensors="pt")
y_val_tokens = tokenizer(y_val, truncation=True, padding=True, return_tensors="pt")

X_test_tokens = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")
y_test_tokens = tokenizer(y_test, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxificationDataset(X_train_tokens, y_train_tokens)
val_dataset = DetoxificationDataset(X_val_tokens, y_val_tokens)
test_dataset = DetoxificationDataset(X_test_tokens, y_test_tokens)


print("Tamaño del dataset de entrenamiento:", len(train_dataset))
print("Ejemplo del dataset:", train_dataset[0])


# training_args = TrainingArguments(
#     output_dir='./checkpoints',
#     eval_strategy="no",
#     per_device_train_batch_size=128,
#     learning_rate=1e-4,
#     num_train_epochs=5,
#     report_to=["none"],
#     #fp16=True, # acelerar entrenaminento
# )


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


['DIGIT year olds can be annoying too , doesn t mean you fight them', 'i am reading early', 'democrats , liberals demonstrate every day that their mental condition is not good.', "Straight people get married and it doesn't always make things better for kids", 'if the uk did anything but talk nonsense and supply weapons id be very surprised', 'and telling nato to leave.', 'Unless we open the can of usa.', 'i enjoy the tasty nice warm pile every morning', 'mind your own stuff', 'i d love to live there if it weren t for california cost of living .', 'The natives will demand billion form the village and get it.', 'ours isnt good either.', 'why are these gangsters still not in jail ?', 'What in all that is?', 'I constructed your house using a bad material', 'It is the only thing keeping these species alive right now, though overhunting has threatened many species.', 'So again, unless you have something useful to say, go away.', 'Bloated nonsense that comes out of these other', 'lockheed or 

## Entrenamiento

In [7]:

model = T5ForConditionalGeneration.from_pretrained(model_name)

num_training_layers = 2 # El número de capas que se entrenarán

# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Descongelar las últimas n capas del decoder
for param in model.decoder.block[-num_training_layers:].parameters():
    param.requires_grad = True

# Mantener la capa de salida (`lm_head`) entrenable
for param in model.lm_head.parameters():
    param.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ESTADÍSTICAS DEL ENTRENAMIENTO:")
print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

ESTADÍSTICAS DEL ENTRENAMIENTO:
Total de parámetros: 76,961,152
Parámetros entrenables: 22,744,064
Parámetros congelados: 54,217,088


In [8]:

training_args = Seq2SeqTrainingArguments(
    output_dir='./checkpoints',
    # evaluation_strategy="no",
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=146*5, # cada 5 epochs, teniendo batches de 128
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-4,
    num_train_epochs=18,
    report_to=["none"], # para no pedir login de 'wandb' y otros
    # fp16=True, # acelera el entrenaminento pero lo empeora
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# epochs = 1

# for _ in range(epochs):
#     # trainer.evaluate()
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
730,2.319400,0.374856
1460,0.391000,0.363603
2190,0.364700,0.361322


TrainOutput(global_step=2232, training_loss=0.8099956751724298, metrics={'train_runtime': 1145.8292, 'train_samples_per_second': 248.126, 'train_steps_per_second': 1.948, 'total_flos': 4851513440317440.0, 'train_loss': 0.8099956751724298, 'epoch': 18.0})

In [15]:
trainer.predict(test_dataset=test_dataset).metrics


{'test_loss': 0.6045551300048828,
 'test_runtime': 2.6657,
 'test_samples_per_second': 187.196,
 'test_steps_per_second': 1.501}

In [11]:
#guardar
model.save_pretrained("detox_model")

In [14]:
#cargar y probar
loaded_model = T5ForConditionalGeneration.from_pretrained("detox_model")
loaded_model.to(device)


frase="who really gives a crap about this ?"

loaded_model.eval()
input_ids = tokenizer(frase, return_tensors="pt").input_ids.to(device)
outputs = loaded_model.generate(input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)



'Who really cares about this?'